In [1]:
#http://alexminnaar.com/2019/08/22/ner-rnns-tensorflow.html
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import pandas as pd
import numpy as np
import os
import time
from sklearn.model_selection import train_test_split

print(tf.__version__)

2.3.3


In [7]:
df = pd.read_csv("./datasets/entities_final.csv")

In [8]:
df.head()

,question,entities
0,who's the writer of Experiment Perilous,"['0', '0', '0', '0', 'B-movie', 'I-movie']"
1,who's the writer of Games,"['0', '0', '0', '0', 'B-movie']"
2,which person wrote The Wolf Man,"['0', '0', '0', 'B-movie', 'I-movie', 'I-movie']"
3,who is the writer of the film Sweet Charity,"['0', '0', '0', '0', '0', '0', '0', 'B-movie',..."
4,who was the writer of Extract,"['0', '0', '0', '0', '0', 'B-movie']"


In [9]:
X_train, X_test, y_train, y_test = train_test_split(
         df["question"], df["entities"], test_size=0.5, random_state=2021)

X_val, X_test, y_val, y_test = train_test_split(
         X_test, y_test, test_size=0.5, random_state=2021)

In [10]:
labels = set([label for line in df["entities"] for label in eval(line)])

In [11]:
all_text = " ".join([" ".join(question) for question in df["question"]])
vocab = sorted(set(all_text))

In [12]:
# create character/id and label/id mapping
char2idx = {u:i+1 for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
label2idx = {u:i+1 for i, u in enumerate(labels)}
idx2label = np.array(labels)

print(idx2label)
print(char2idx)

{'I-director', 'I-actor', 'B-director', 'B-actor', '0', 'B-movie', 'I-tag', 'I-movie', 'B-tag', 'B-writer', 'I-writer'}
{' ': 1, '!': 2, "'": 3, '(': 4, ')': 5, '+': 6, ',': 7, '-': 8, '.': 9, '/': 10, '0': 11, '1': 12, '2': 13, '3': 14, '4': 15, '5': 16, '6': 17, '7': 18, '8': 19, '9': 20, 'A': 21, 'B': 22, 'C': 23, 'D': 24, 'E': 25, 'F': 26, 'G': 27, 'H': 28, 'I': 29, 'J': 30, 'K': 31, 'L': 32, 'M': 33, 'N': 34, 'O': 35, 'P': 36, 'Q': 37, 'R': 38, 'S': 39, 'T': 40, 'U': 41, 'V': 42, 'W': 43, 'X': 44, 'Y': 45, 'Z': 46, 'a': 47, 'b': 48, 'c': 49, 'd': 50, 'e': 51, 'f': 52, 'g': 53, 'h': 54, 'i': 55, 'j': 56, 'k': 57, 'l': 58, 'm': 59, 'n': 60, 'o': 61, 'p': 62, 'q': 63, 'r': 64, 's': 65, 't': 66, 'u': 67, 'v': 68, 'w': 69, 'x': 70, 'y': 71, 'z': 72, '¡': 73, '½': 74, 'À': 75, 'É': 76, 'Õ': 77, 'à': 78, 'á': 79, 'â': 80, 'ã': 81, 'ä': 82, 'å': 83, 'æ': 84, 'ç': 85, 'è': 86, 'é': 87, 'ê': 88, 'ì': 89, 'í': 90, 'î': 91, 'ï': 92, 'ñ': 93, 'ò': 94, 'ó': 95, 'ô': 96, 'ö': 97, 'ø': 98, 'û': 9

In [13]:
print(X_train.iloc[0])

str(['0', '0', '0', 'B-movie', '0'])

what language is Watchmen in


"['0', '0', '0', 'B-movie', '0']"

In [15]:
def split_char_labels(X, y):
    '''
    For a given input/output example, break tokens into characters while keeping 
    the same label.
    '''

    tokens = X
    labels = eval(y)

    input_chars = []
    output_char_labels = []
    print(tokens)
    print(labels)
    print()
    for token,label in zip(tokens,labels):
        
        input_chars.extend([char for char in token])
        input_chars.extend(' ')
        output_char_labels.extend([label]*len(token))
        output_char_labels.extend('0')
        
#     print(input_chars)
#     print(output_char_labels)
    
#     input()
    
    
    return [[char2idx[x] for x in input_chars[:-1]],np.array([label2idx[x] for x in output_char_labels[:-1]])]

train_formatted = list(map(split_char_labels, X_train, y_train))
test_formatted = list(map(split_char_labels, X_test, y_test))
valid_formatted = list(map(split_char_labels, X_val, y_val))

print(len(train_formatted))
print(len(test_formatted))
print(len(valid_formatted))

what language is Watchmen in
['0', '0', '0', 'B-movie', '0']

who directed Hollywood Shuffle
['0', '0', 'B-movie', 'I-movie']

what movies was Marjorie David the writer of
['0', '0', '0', 'B-writer', 'I-writer', '0', '0', '0']

when was the film Young Cassidy released
['0', '0', '0', '0', 'B-movie', 'I-movie', '0']

who stars in When the Bough Breaks
['0', '0', '0', 'B-movie', 'I-movie', 'I-movie', 'I-movie']

who directed the film Anne of the Thousand Days
['0', '0', 'B-movie', '0', 'I-movie', 'I-movie', 'I-movie', 'I-movie', 'I-movie']

what does Raymond Bussières star in
['0', '0', 'B-actor', 'I-actor', '0', '0']

Cousin Bette, is it any good
['B-movie', '0', '0', '0', '0', '0']

what was the release year of the movie Many Rivers to Cross
['0', '0', '0', '0', '0', '0', '0', '0', 'B-movie', 'I-movie', 'I-movie', 'I-movie']

what is a movie written by Clare Boothe Luce
['0', '0', '0', '0', '0', '0', 'B-writer', 'I-writer', 'I-writer']

which movies are about beauty pageant
['0', '0', 

what are some words that  describe movie 2 Fast 2 Furious
['0', '0', '0', '0', '0', '0', '0', '0', 'B-movie', 'I-movie', 'I-movie', 'I-movie']

can you describe movie The Ramen Girl in a few words
['0', '0', '0', '0', 'B-movie', 'I-movie', 'I-movie', '0', '0', '0', '0']

Stephen Weeks was the director of which films
['B-director', 'I-director', '0', '0', '0', '0', '0', '0']

what is a film written by Pat E. Johnson
['0', '0', '0', '0', '0', '0', 'B-writer', 'I-writer', 'I-writer']

what movies are about alberto cavalcanti
['0', '0', '0', '0', 'B-tag', 'I-tag']

what was the release date of the film Kicking and Screaming
['0', '0', '0', '0', '0', '0', '0', '0', 'B-movie', 'I-movie', 'I-movie']

what was the genre of the movie Gator
['0', '0', '0', '0', '0', '0', '0', 'B-movie']

what type of film is The Poughkeepsie Tapes
['0', '0', '0', '0', '0', 'B-movie', 'I-movie', 'I-movie']

Joe Ranft was the director on which movies
['B-director', 'I-director', '0', '0', '0', '0', '0', '0']

what

who is the author of 8 Heads in a Duffel Bag
['0', '0', '0', '0', '0', 'B-movie', 'I-movie', 'I-movie', 'I-movie', 'I-movie', 'I-movie']

who is the director that directed The Gift
['0', '0', '0', '0', '0', '0', 'B-movie', 'I-movie']

what was the release date of the movie The Battle of China
['0', '0', '0', '0', '0', 'B-movie', '0', '0', 'I-movie', 'I-movie', 'I-movie', 'I-movie']

what is the genre for the film Erin Brockovich
['0', '0', '0', '0', '0', '0', '0', 'B-movie', 'I-movie']

who's the writer of The Delta Force
['0', '0', '0', '0', 'B-movie', 'I-movie', 'I-movie']

which words describe Aria
['0', '0', '0', 'B-movie']

what does W.C. Fields star in
['0', '0', 'B-actor', 'I-actor', '0', '0']

what movie did Steven Brill write the story for
['0', '0', '0', 'B-writer', 'I-writer', '0', '0', '0', '0']

Alain Guiraudie directed which films
['B-director', 'I-director', '0', '0', '0']

what type of film is Houseguest
['0', '0', '0', '0', '0', 'B-movie']

what year was the movie Some

which person directed Yolanda and the Thief
['0', '0', '0', 'B-movie', 'I-movie', 'I-movie', 'I-movie']

the film Hamlet was directed by who
['0', '0', 'B-movie', '0', '0', '0', '0']

who in the world wrote Apartment 1303
['0', '0', '0', '0', '0', 'B-movie', 'I-movie']

what films did Elissa Down direct
['0', '0', '0', 'B-director', 'I-director', '0']

what was the release year of Big Miracle
['0', '0', '0', '0', '0', '0', 'B-movie', 'I-movie']

who starred in A Moment of Innocence
['0', '0', '0', 'B-movie', 'I-movie', 'I-movie', 'I-movie']

what movies did Gemma Jones act in
['0', '0', '0', 'B-actor', 'I-actor', '0', '0']

what films was Rodrigo Bellott a writer on
['0', '0', '0', 'B-writer', 'I-writer', '0', '0', '0']

what genre of film is Love Wrecked
['0', '0', '0', '0', '0', 'B-movie', 'I-movie']

what films was Philip Goodhew a writer on
['0', '0', '0', 'B-writer', 'I-writer', '0', '0', '0']

which person wrote the movie Jackass Number Two
['0', '0', '0', '0', '0', 'B-movie', 'I


what year was the film The Beast released
['0', '0', '0', '0', '0', 'B-movie', 'I-movie', '0']

Luke Bracey appears in which movies
['B-actor', 'I-actor', '0', '0', '0', '0']

what film did Scot McFadyen write
['0', '0', '0', 'B-writer', 'I-writer', '0']

the movie Guess Who's Coming to Dinner starred who
['0', '0', 'B-movie', 'I-movie', 'I-movie', 'I-movie', 'I-movie', '0', '0']

what was the release year of the movie I Love You, I Love You Not
['0', '0', '0', '0', '0', '0', '0', '0', 'B-movie', 'I-movie', 'I-movie', 'I-movie', 'I-movie', 'I-movie', 'I-movie']

what is the genre for the film Selma
['0', '0', '0', '0', '0', '0', '0', 'B-movie']

what is the genre for the film Bastards
['0', '0', '0', '0', '0', '0', '0', 'B-movie']

what genre of film is Trouble in Paradise
['0', '0', '0', '0', '0', 'B-movie', 'I-movie', 'I-movie']

who is the director for The Mysterious Island
['0', '0', '0', '0', '0', 'B-movie', 'I-movie', 'I-movie']

what was the release year of the movie Kind Lady


...

what films are about genndy tartakovsky
['0', '0', '0', '0', 'B-tag', 'I-tag']

Bill Douglas was the director of which movies
['B-director', 'I-director', '0', '0', '0', '0', '0', '0']

what genre is Rob the Mob in
['0', '0', '0', 'B-movie', 'I-movie', 'I-movie', '0']

what does Franciszek Pieczka star in
['0', '0', 'B-actor', 'I-actor', '0', '0']

what movies can be described with peter greenaway
['0', '0', '0', '0', '0', '0', 'B-tag', 'I-tag']

what rating would people give You've Got Mail
['0', '0', '0', '0', '0', 'B-movie', 'I-movie', 'I-movie']

the director of The Legend of Suram Fortress was
['0', '0', 'B-movie', 'I-movie', 'I-movie', 'I-movie', 'I-movie', 'I-movie', '0']

which screenwriter wrote the script for Waiting to Exhale
['0', '0', '0', '0', '0', '0', 'B-movie', 'I-movie', 'I-movie']

who are the actors in The Time Traveler's Wife
['0', '0', '0', '0', '0', 'B-movie', 'I-movie', 'I-movie', 'I-movie']

what movies did Kern Saxton write
['0', '0', '0', 'B-writer', 'I-write

what films did Regina King act in
['0', '0', '0', 'B-actor', 'I-actor', '0', '0']

what was the release date of the film The Color of Paradise
['0', '0', '0', '0', '0', 'B-movie', '0', '0', 'I-movie', 'I-movie', 'I-movie', 'I-movie']

which person wrote the movie Uprising
['0', '0', '0', '0', '0', 'B-movie']

who was the director of Everybody Wants to Be Italian
['0', '0', '0', '0', '0', 'B-movie', 'I-movie', 'I-movie', 'I-movie', 'I-movie']

which movies are about river phoenix
['0', '0', '0', '0', 'B-tag', 'I-tag']

who was Odd Man Out directed by
['0', '0', 'B-movie', 'I-movie', 'I-movie', '0', '0']

who's the director of Captain EO
['0', '0', '0', '0', 'B-movie', 'I-movie']

the film Pitch Black was directed by who
['0', '0', 'B-movie', 'I-movie', '0', '0', '0', '0']

who directed the movie Bachelor Mother
['0', '0', '0', '0', 'B-movie', 'I-movie']

what is the genre of the film I See a Dark Stranger
['0', '0', '0', '0', '0', '0', '0', 'B-movie', 'I-movie', 'I-movie', 'I-movie', 'I


what terms are applicable to The House on Telegraph Hill
['0', '0', '0', '0', '0', 'B-movie', 'I-movie', 'I-movie', 'I-movie', 'I-movie']

what does Penélope Cruz star in
['0', '0', 'B-actor', 'I-actor', '0', '0']

who directed Stepmom
['0', '0', 'B-movie']

Wild Geese II is a film written by this person
['B-movie', 'I-movie', 'I-movie', '0', '0', '0', '0', '0', '0', '0']

when was the film How to Stuff a Wild Bikini released
['0', '0', '0', '0', 'B-movie', 'I-movie', 'I-movie', 'I-movie', 'I-movie', 'I-movie', '0']

who is the writer of Rocky III
['0', '0', '0', '0', '0', 'B-movie', 'I-movie']

when was Ten released
['0', '0', 'B-movie', '0']

who was the director of Airport '77
['0', '0', '0', '0', '0', 'B-movie', 'I-movie']

who is the director for Hello Herman
['0', '0', '0', '0', '0', 'B-movie', 'I-movie']

what genre of film is Road to Zanzibar
['0', '0', '0', '0', '0', 'B-movie', 'I-movie', 'I-movie']

what was the genre of The Triplets of Belleville
['0', '0', '0', '0', 'B-mov

['B-movie', 'I-movie', 'I-movie', '0', '0', '0', '0', '0']

what films did Rick Shaughnessy write
['0', '0', '0', 'B-writer', 'I-writer', '0']

what is the genre of the movie Haunt
['0', '0', '0', '0', '0', '0', '0', 'B-movie']

which films can be described by glam rock
['0', '0', '0', '0', '0', '0', 'B-tag', 'I-tag']

what is the genre for Nina's Heavenly Delights
['0', '0', '0', '0', '0', 'B-movie', 'I-movie', 'I-movie']

what genre of film is The Opposite Sex
['0', '0', '0', '0', '0', 'B-movie', 'I-movie', 'I-movie']

what year was Insomnia released
['0', '0', '0', 'B-movie', '0']

who acted in the movie Off the Black
['0', '0', '0', 'B-movie', '0', 'I-movie', 'I-movie', 'I-movie']

who wrote Thunderpants
['0', '0', 'B-movie']

who is the director for Home Alone 4
['0', '0', '0', '0', '0', 'B-movie', 'I-movie', 'I-movie']

who acted in Sitting Pretty
['0', '0', '0', 'B-movie', 'I-movie']

who was Great Expectations directed by
['0', '0', 'B-movie', 'I-movie', '0', '0']

what film di

['0', '0', '0', '0', 'B-movie']

the film He Said, She Said starred which actors
['0', '0', 'B-movie', 'I-movie', 'I-movie', 'I-movie', '0', '0', '0']

Shriek If You Know What I Did Last Friday the Thirteenth, when was it released
['B-movie', 'I-movie', 'I-movie', 'I-movie', 'I-movie', 'I-movie', 'I-movie', 'I-movie', 'I-movie', 'I-movie', '0', '0', '0', '0', '0']

who is listed as director for Went the Day Well
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0']

which movies can be described by michael nyqvist
['0', '0', '0', '0', '0', '0', 'B-tag', 'I-tag']

when was Danton released
['0', '0', 'B-movie', '0']

what was the release date of the movie Jennifer's Body
['0', '0', '0', '0', '0', '0', '0', '0', 'B-movie', 'I-movie']

what movies are about brian cox
['0', '0', '0', '0', 'B-tag', 'I-tag']

what words describe film Real Men
['0', '0', '0', '0', 'B-movie', 'I-movie']

what films did Thomas Ian Griffith write
['0', '0', '0', 'B-writer', 'I-writer', 'I-writer', '0']

the movie Th

In [16]:
len(train_formatted[1][1])

7

In [17]:
train_formatted[0]

[[69, 1, 54, 1, 47, 1, 66, 1, 1], array([5, 5, 5, 5, 5, 5, 6, 5, 5])]

In [18]:
# training generator
def gen_train_series():
    for eg in train_formatted:
        yield eg[0],eg[1]
    
# validation generator
def gen_valid_series():
    for eg in valid_formatted:
        yield eg[0],eg[1]
    
# test generator
def gen_test_series():
    for eg in test_formatted:
        yield eg[0],eg[1]
      
# create Dataset objects for train, test and validation sets  
series = tf.data.Dataset.from_generator(gen_train_series,output_types=(tf.int32, tf.int32),output_shapes = ((None, None)))
series_valid = tf.data.Dataset.from_generator(gen_valid_series,output_types=(tf.int32, tf.int32),output_shapes = ((None, None)))
series_test = tf.data.Dataset.from_generator(gen_test_series,output_types=(tf.int32, tf.int32),output_shapes = ((None, None)))

BATCH_SIZE = 128
BUFFER_SIZE=1000
    
# create padded batch series objects for train, test and validation sets
ds_series_batch = series.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes=([None], [None]), drop_remainder=True)
ds_series_batch_valid = series_valid.padded_batch(BATCH_SIZE, padded_shapes=([None], [None]), drop_remainder=True)
ds_series_batch_test = series_test.padded_batch(BATCH_SIZE, padded_shapes=([None], [None]), drop_remainder=True)

# print example batches
for input_example_batch, target_example_batch in ds_series_batch_valid.take(1):
    print(input_example_batch)
    print(target_example_batch)


tf.Tensor(
[[69  1 54 ...  0  0  0]
 [69  1 54 ...  0  0  0]
 [69  1 54 ...  0  0  0]
 ...
 [69  1 54 ...  0  0  0]
 [69  1 54 ...  0  0  0]
 [69  1 54 ...  0  0  0]], shape=(128, 27), dtype=int32)
tf.Tensor(
[[5 5 5 ... 0 0 0]
 [5 5 5 ... 0 0 0]
 [5 5 5 ... 0 0 0]
 ...
 [5 5 5 ... 0 0 0]
 [5 5 5 ... 0 0 0]
 [5 5 5 ... 0 0 0]], shape=(128, 27), dtype=int32)


In [11]:
vocab_size = len(vocab)+1

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

label_size = len(labels)  

# build LSTM model
def build_model(vocab_size,label_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                          batch_input_shape=[batch_size, None],mask_zero=True),
        tf.keras.layers.LSTM(rnn_units,
                    return_sequences=True, # a saida da lstm vai ser um conj de vetores
                    stateful=True,
                    recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(label_size)
        ])
    return model

model = build_model(
    vocab_size = len(vocab)+1,
    label_size=len(labels)+1,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 256)          25856     
_________________________________________________________________
lstm (LSTM)                  (128, None, 1024)         5246976   
_________________________________________________________________
dense (Dense)                (128, None, 12)           12300     
Total params: 5,285,132
Trainable params: 5,285,132
Non-trainable params: 0
_________________________________________________________________


In [12]:
import os

# define loss function
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss, metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [13]:
EPOCHS=20
history = model.fit(ds_series_batch, epochs=EPOCHS, validation_data=ds_series_batch_valid,callbacks=[checkpoint_callback])

Epoch 1/20
39/39 [==============================] - 31s 792ms/step - loss: 0.4897 - sparse_categorical_accuracy: 0.7956 - val_loss: 0.3467 - val_sparse_categorical_accuracy: 0.8234
Epoch 2/20
39/39 [==============================] - 35s 897ms/step - loss: 0.3208 - sparse_categorical_accuracy: 0.8251 - val_loss: 0.2699 - val_sparse_categorical_accuracy: 0.8365
Epoch 3/20
39/39 [==============================] - 37s 943ms/step - loss: 0.2617 - sparse_categorical_accuracy: 0.8430 - val_loss: 0.2351 - val_sparse_categorical_accuracy: 0.8517
Epoch 4/20
39/39 [==============================] - 38s 966ms/step - loss: 0.2314 - sparse_categorical_accuracy: 0.8480 - val_loss: 0.2172 - val_sparse_categorical_accuracy: 0.8576
Epoch 5/20
39/39 [==============================] - 37s 947ms/step - loss: 0.2185 - sparse_categorical_accuracy: 0.8512 - val_loss: 0.2027 - val_sparse_categorical_accuracy: 0.8579
Epoch 6/20
39/39 [==============================] - 39s 988ms/step - loss: 0.2065 - sparse_cate

In [14]:
from sklearn.metrics import classification_report, confusion_matrix

preds = np.array([])
y_trues= np.array([])

# iterate through test set, make predictions based on trained model
for input_example_batch, target_example_batch in ds_series_batch_test:

    pred=model.predict_on_batch(input_example_batch)
    pred_max=tf.argmax(tf.nn.softmax(pred),2).numpy().flatten()
    y_true=target_example_batch.numpy().flatten()

    preds=np.concatenate([preds,pred_max])
    y_trues=np.concatenate([y_trues,y_true])

# remove padding from evaluation
remove_padding = [(p,y) for p,y in zip(preds,y_trues) if y!=0]

r_p = [x[0] for x in remove_padding]
r_t = [x[1] for x in remove_padding]

# print confusion matrix and classification report
print(confusion_matrix(r_p,r_t))
print(classification_report(r_p,r_t))

[[27608   821   690    82    73    43   125   224   207   220    81]
 [  135   705   255     0     0     7    34     0    10     0    16]
 [  462   155  1946     0     0    11     6     0     0     2     0]
 [    0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0]
 [   10     0     6     0     0    40     0     0     0    22     0]
 [   56    27    56    16     0    35    91     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     0     0     1     0]
 [    6     7     0     0     0     0     0     0    17     0    28]]
              precision    recall  f1-score   support

         1.0       0.98      0.91      0.94     30174
         2.0       0.41      0.61      0.49      1162
         3.0       0.66      0.75      0.70      2582
         4.0       0.00

/usr/local/Caskroom/miniconda/base/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/Caskroom/miniconda/base/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/Caskroom/miniconda/base/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [3]:
import tempfile

MODEL_DIR = tempfile.gettempdir()
version = 1
export_path = os.path.join('./save', str(version))
print('export_path = {}\n'.format(export_path))

tf.keras.models.save_model(
    model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None,
)

print('\nSaved model:')
!ls -l {export_path}

export_path = ./save/1



NameError: name 'model' is not defined

In [4]:
reconstructed_model = tf.keras.models.load_model("./save/1", compile=False)

In [5]:
!ls

Wikimovies entities with BERT.ipynb generate_csv.ipynb
Wikimovies entities with LSTM.ipynb save
datasets                            training_checkpoints


In [27]:
input_str = "What films did Guillermo del Toro directed?"
input_str = input_str[0].lower() + input_str[1:]
if input_str[-1] == '?': 
    input_str = input_str[:-1]
    

inf_X = pd.Series([input_str] + ([""] * 127))
inf_y = pd.Series([str(["0" for _ in range(len(input_str.split()))]) for _ in range(128)])

inf_formatted = list(map(split_char_labels, inf_X, inf_y))

what films did Guillermo del Toro directed
['0', '0', '0', '0', '0', '0', '0']


['0', '0', '0', '0', '0', '0', '0']


['0', '0', '0', '0', '0', '0', '0']


['0', '0', '0', '0', '0', '0', '0']


['0', '0', '0', '0', '0', '0', '0']


['0', '0', '0', '0', '0', '0', '0']


['0', '0', '0', '0', '0', '0', '0']


['0', '0', '0', '0', '0', '0', '0']


['0', '0', '0', '0', '0', '0', '0']


['0', '0', '0', '0', '0', '0', '0']


['0', '0', '0', '0', '0', '0', '0']


['0', '0', '0', '0', '0', '0', '0']


['0', '0', '0', '0', '0', '0', '0']


['0', '0', '0', '0', '0', '0', '0']


['0', '0', '0', '0', '0', '0', '0']


['0', '0', '0', '0', '0', '0', '0']


['0', '0', '0', '0', '0', '0', '0']


['0', '0', '0', '0', '0', '0', '0']


['0', '0', '0', '0', '0', '0', '0']


['0', '0', '0', '0', '0', '0', '0']


['0', '0', '0', '0', '0', '0', '0']


['0', '0', '0', '0', '0', '0', '0']


['0', '0', '0', '0', '0', '0', '0']


['0', '0', '0', '0', '0', '0', '0']


['0', '0', '0', '0', '0', '0', '0']


['0', '

In [28]:
inf_formatted[0]

[[69, 1, 54, 1, 47, 1, 66, 1, 1, 1, 52, 1, 55],
 array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])]

In [29]:
def gen_inf_series():
    for eg in inf_formatted:
        yield eg[0],eg[1]

series_inf = tf.data.Dataset.from_generator(gen_inf_series, output_types=(tf.int32, tf.int32), output_shapes=((None, None)))

BATCH_SIZE = 128
BUFFER_SIZE=1000

ds_series_batch_inf = series_inf.padded_batch(BATCH_SIZE, padded_shapes=([None], [None]), drop_remainder=True)


In [30]:
X, y = next(iter(ds_series_batch_inf))

In [31]:
X[0]

<tf.Tensor: shape=(13,), dtype=int32, numpy=array([69,  1, 54,  1, 47,  1, 66,  1,  1,  1, 52,  1, 55], dtype=int32)>

In [32]:
pred = reconstructed_model.predict_on_batch(X)
pred_max = tf.argmax(tf.nn.softmax(pred),2).numpy()

In [33]:
pred_max[0]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])